In [29]:
import csv
import requests
import numpy as np
import pandas as pd
from io import StringIO

In [30]:
na_values = ""
column_types = {
    "Criteria ID": np.int32, "Name": str, "Canonical Name": str, "Parent ID": str, 
    "Country Code": str, "Target Type": str, "status": str
}
response = requests.get("https://developers.google.com/adwords/api/docs/appendix/geo/geotargets-2019-08-14.csv")
csv_file = StringIO(response.content.decode("utf-8"))
geo_targets = pd.read_csv(csv_file, sep=",", na_values=na_values, dtype=column_types)

In [31]:
geo_targets.shape

(102029, 7)

In [32]:
# 处理 缺失值, 用 0 填充
geo_targets['Parent ID'] = geo_targets['Parent ID'].apply(lambda x: int(x) if not isinstance(x, float) else 0).astype(np.int32)

In [33]:
geo_targets.head()

,Criteria ID,Name,Canonical Name,Parent ID,Country Code,Target Type,Status
0,1000002,Kabul,"Kabul,Kabul,Afghanistan",9075393,AF,City,Active
1,1000003,Luanda,"Luanda,Luanda Province,Angola",9070431,AO,City,Active
2,1000004,The Valley,"The Valley,Anguilla",2660,AI,City,Active
3,1000010,Abu Dhabi,"Abu Dhabi,Abu Dhabi,United Arab Emirates",9041082,AE,City,Active
4,1000011,Ajman,"Ajman,Ajman,United Arab Emirates",9047096,AE,City,Active


In [34]:
# filter country
geo_country = geo_targets[geo_targets['Parent ID'] == 0]

In [35]:
geo_country.dtypes

Criteria ID        int32
Name              object
Canonical Name    object
Parent ID          int32
Country Code      object
Target Type       object
Status            object
dtype: object

In [36]:
# 查看占用内存
geo_targets.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102029 entries, 0 to 102028
Data columns (total 7 columns):
Criteria ID       102029 non-null int32
Name              102029 non-null object
Canonical Name    102029 non-null object
Parent ID         102029 non-null int32
Country Code      102013 non-null object
Target Type       102029 non-null object
Status            102029 non-null object
dtypes: int32(2), object(5)
memory usage: 33.8 MB


In [18]:
# 查看 平均每个数据类型的大小
for dtype in ['object','int']:
    selected_dtype = geo_targets.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.5f} MB".format(dtype, mean_usage_mb))

Average memory usage for object columns: 5.49675 MB
Average memory usage for int columns: 0.00012 MB


In [10]:
max(geo_targets['Criteria ID'])  # 看来得用int32

9100164

In [11]:
int_types = ["uint8", "int8", "int16", "int32"]
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for int32
---------------------------------------------------------------
min = -2147483648
max = 2147483647
---------------------------------------------------------------



In [19]:
geo_targets['Country Code'].describe()

count     102013
unique       239
top           US
freq       55422
Name: Country Code, dtype: object

In [25]:
geo_targets['Status'].describe()

count     102029
unique         2
top       Active
freq      102026
Name: Status, dtype: object

In [20]:
country_code_category = geo_targets['Country Code'].astype('category')

In [23]:
country_code_category.cat.codes

0           2
1           7
2           4
3           1
4           1
         ... 
102024    170
102025    170
102026    221
102027    221
102028     64
Length: 102029, dtype: int16

In [24]:
# 将枚举的变量转为 categorical
catagorical_geo_targets = pd.DataFrame()
for col in geo_targets.columns:
    num_unique_values = len(geo_targets[col].unique())
    num_total_values = len(geo_targets[col])
    if num_unique_values / num_total_values < 0.5:
        print("Categorical column: ", col)
        catagorical_geo_targets.loc[:,col] = geo_targets[col].astype('category')
    else:
        catagorical_geo_targets.loc[:,col] = geo_targets[col]

Categorical column:  Parent ID
Categorical column:  Country Code
Categorical column:  Target Type
Categorical column:  Status


In [28]:
catagorical_geo_targets.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102029 entries, 0 to 102028
Data columns (total 7 columns):
Criteria ID       102029 non-null int32
Name              102029 non-null object
Canonical Name    102029 non-null object
Parent ID         102029 non-null category
Country Code      102013 non-null category
Target Type       102029 non-null category
Status            102029 non-null category
dtypes: category(4), int32(1), object(2)
memory usage: 15.9 MB


In [40]:
china_geo = catagorical_geo_targets[catagorical_geo_targets['Country Code'] == 'CN']
america_geo = catagorical_geo_targets[catagorical_geo_targets['Country Code'] == 'US']

In [43]:
china_geo.head()

,Criteria ID,Name,Canonical Name,Parent ID,Country Code,Target Type,Status
2477,1003334,Beijing,"Beijing,Beijing,China",20163,CN,City,Active
2478,1003338,Tianjin,"Tianjin,Tianjin,China",20164,CN,City,Active
2479,1003339,Chengde,"Chengde,Hebei,China",20165,CN,City,Active
2480,1003341,Handan,"Handan,Hebei,China",20165,CN,City,Active
2481,1003342,Hengshui,"Hengshui,Hebei,China",20165,CN,City,Active


In [44]:
america_geo.head()

,Criteria ID,Name,Canonical Name,Parent ID,Country Code,Target Type,Status
10598,1012873,Anchorage,"Anchorage,Anchorage,Alaska,United States",21132,US,City,Active
10599,1012874,Anderson,"Anderson,Alaska,United States",21132,US,City,Active
10600,1012875,Angoon,"Angoon,Alaska,United States",21132,US,City,Active
10601,1012876,Atqasuk,"Atqasuk,Alaska,United States",21132,US,City,Active
10602,1012877,Utqiagvik,"Utqiagvik,Alaska,United States",21132,US,City,Active
